# Creating supercells with pymatgen

The [Pymatgen](https://pymatgen.org/) python library allows to setup
solid-state calculations using a flexible set of classes as well as an API
to an online data base of structures. Its `Structure` and `Lattice`
objects are directly supported by the DFTK `load_atoms` and `load_lattice`
functions, such that DFTK may be readily used to run calculation on systems
defined in pymatgen. Using the `pymatgen_structure` function a conversion
from DFTK to pymatgen structures is also possible. In the following we
use this to create a silicon supercell and find its LDA ground state
using direct minimisation. To run this example Julia's `PyCall` package
needs to be able to find an installation of `pymatgen`.

First we setup the silicon lattice in DFTK.

In [1]:
using DFTK

a = 10.263141334305942  # Lattice constant in Bohr
lattice = a / 2 .* [[0 1 1.]; [1 0 1.]; [1 1 0.]]
Si = ElementPsp(:Si, psp=load_psp("hgh/lda/Si-q4"))
atoms = [Si => [ones(3)/8, -ones(3)/8]];

Next we make a `[2, 2, 2]` supercell using pymatgen

In [2]:
pystruct = pymatgen_structure(lattice, atoms)
pystruct.make_supercell([2, 2, 2])
lattice = load_lattice(pystruct)
atoms = [Si => [s.frac_coords for s in pystruct.sites]];

┌ Warning: `vendor()` is deprecated, use `BLAS.get_config()` and inspect the output instead
│   caller = npyinitialize() at numpy.jl:67
└ @ PyCall /home/runner/.julia/packages/PyCall/L0fLP/src/numpy.jl:67


Setup an LDA model and discretize using
a single k-point and a small `Ecut` of 5 Hartree.

In [3]:
model = model_LDA(lattice, atoms)
basis = PlaneWaveBasis(model; Ecut=5, kgrid=(1, 1, 1))

PlaneWaveBasis discretization:
    Ecut                 : 5.0 Ha
    fft_size             : (32, 32, 32), 32768 total points
    kgrid type           : Monkhorst-Pack
    kgrid                : [1, 1, 1]
    num. irred. kpoints  : 1

    Discretized Model(lda_xc_teter93, 3D):
        lattice (in Bohr)    : [0         , 10.2631   , 10.2631   ]
                               [10.2631   , 0         , 10.2631   ]
                               [10.2631   , 10.2631   , 0         ]
        unit cell volume     : 2162.1 Bohr³
    
        atoms                : Si₁₆
        atom potentials      : ElementPsp(Si, psp="hgh/lda/si-q4")
    
        num. electrons       : 64
        spin polarization    : none
        temperature          : 0 Ha
    
        terms                : Kinetic()
                               AtomicLocal()
                               AtomicNonlocal()
                               Ewald()
                               PspCorrection()
                               

Find the ground state using direct minimisation (always using SCF is boring ...)

In [4]:
scfres = direct_minimization(basis, tol=1e-5);

Iter     Function value   Gradient norm 
     0     1.119351e+02     1.614488e+00
 * time: 0.5037741661071777
     1     1.147571e+01     9.612708e-01
 * time: 2.0165162086486816
     2    -1.153692e+01     1.104480e+00
 * time: 2.1254541873931885
     3    -3.377962e+01     7.456426e-01
 * time: 2.237626075744629
     4    -4.748166e+01     5.609969e-01
 * time: 2.373444080352783
     5    -5.689246e+01     1.885948e-01
 * time: 2.4775431156158447
     6    -5.979406e+01     1.423394e-01
 * time: 2.5520541667938232
     7    -6.094395e+01     5.759049e-02
 * time: 2.6468851566314697
     8    -6.141515e+01     4.136405e-02
 * time: 2.7207460403442383
     9    -6.166655e+01     4.903475e-02
 * time: 2.792717218399048
    10    -6.185895e+01     2.626912e-02
 * time: 2.8649160861968994
    11    -6.199355e+01     1.884512e-02
 * time: 2.9460361003875732
    12    -6.206902e+01     1.646119e-02
 * time: 3.019599199295044
    13    -6.211331e+01     1.383126e-02
 * time: 3.09132909774780

In [5]:
scfres.energies

Energy breakdown (in Ha):
    Kinetic             25.7671072
    AtomicLocal         -18.8557750
    AtomicNonlocal      14.8522704
    Ewald               -67.1831486
    PspCorrection       -2.3569765
    Hartree             4.8485386 
    Xc                  -19.3336825

    total               -62.261666453753